In [48]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import string
import re
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
# Preprocessing allows us to standarsize our data
from sklearn import preprocessing
# Allows us to split our data into training and testing data
from sklearn.model_selection import train_test_split
# Allows us to test parameters of classification algorithms and find the best one
from sklearn.model_selection import GridSearchCV
# Logistic Regression classification algorithm
from sklearn.linear_model import LogisticRegression
# Support Vector Machine classification algorithm
from sklearn.svm import SVC
# Decision Tree classification algorithm
from sklearn.tree import DecisionTreeClassifier
# K Nearest Neighbors classification algorithm
from sklearn.neighbors import KNeighborsClassifier

In [50]:
from wordcloud import WordCloud

In [51]:
train=pd.read_csv('twittertrain.csv')
test=pd.read_csv('twittertest.csv')

In [52]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [53]:
print(train.shape)
test.shape

(7613, 5)


(3263, 4)

In [54]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [55]:
train=train.drop(columns=['id','keyword','location'])
train

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [56]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [57]:
feat=train[['text']]
label=train[['target']]

In [58]:
ps=PorterStemmer()
wordnet=WordNetLemmatizer()

In [59]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')

In [61]:
def preprocessing(data):
    corpus=[]
    for i in range(len(data)):
        #remove urls
        tweet=re.sub(r'http\S+',' ',data[i])
        
        #remove html tags
        tweet=re.sub(r'<.*?>',' ',tweet)
        tweet=re.sub('@\w+([-.]\w+)*',' ',tweet)
        tweet=re.sub('&\w+([-.]\w+)*',' ',tweet)
        
        #remove digits
        tweet=re.sub(r'\d+',' ',tweet)
        
        #remove hashtags
        tweet=re.sub(r'#\w+',' ',tweet)
        #remove punctuations and numbers
        review=re.sub('[^a-zA-Z]',' ',tweet)
        review=review.lower()
        review=review.split()
        
        #review=expand_contractions(review)
        lemma=WordNetLemmatizer()
        review=[lemma.lemmatize(word) for word in review if word not in all_stopwords]
        review=' '.join(review)
        corpus.append(review)
    return corpus

In [62]:
feat.head()

,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...


In [63]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [64]:
feat['text']=preprocessing(feat['text'])

<ipython-input-64-1297f3608949>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feat['text']=preprocessing(feat['text'])


In [65]:
feat.head()

,text
0,deed reason may allah forgive u
1,forest fire near la ronge sask canada
2,resident asked shelter place notified officer ...
3,people receive evacuation order california
4,got sent photo ruby smoke pours school


In [46]:
#Model Building
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [66]:
#preprocessing the text data
test=test.drop(columns=['id','keyword','location'])
test

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


In [67]:
test['text']=preprocessing(test['text'])
test.head()

,text
0,happened terrible car crash
1,heard different city stay safe everyone
2,forest fire spot pond goose fleeing across str...
3,apocalypse lighting
4,typhoon soudelor kill china taiwan


In [68]:
test.shape

(3263, 1)

In [69]:
import gensim

In [70]:
a=feat['text']
b=test['text']

In [72]:
sentence=pd.concat([a,b],axis=0)

In [28]:
sentence.head(5)

0                      deed reason may allah forgive u
1                forest fire near la ronge sask canada
2    resident asked shelter place notified officer ...
3           people receive evacuation order california
4               got sent photo ruby smoke pours school
Name: text, dtype: object

In [73]:
print(len(sentence))

10876


In [74]:
print(type(sentence))
print(sentence.shape)
print(sentence)

<class 'pandas.core.series.Series'>
(10876,)
0                         deed reason may allah forgive u
1                   forest fire near la ronge sask canada
2       resident asked shelter place notified officer ...
3              people receive evacuation order california
4                  got sent photo ruby smoke pours school
                              ...                        
3258    earthquake safety los angeles safety fastener ...
3259    storm ri worse last hurricane city others hard...
3260                        green line derailment chicago
3261              meg issue hazardous weather outlook hwo
3262                   activated municipal emergency plan
Name: text, Length: 10876, dtype: object


In [75]:
sentence.iloc[1]

'forest fire near la ronge sask canada'

In [76]:
'forest' in sentence.iloc[1]

True

In [84]:
sentence2= sentence.apply(lambda x: x.split())
sentence2

0                  [deed, reason, may, allah, forgive, u]
1           [forest, fire, near, la, ronge, sask, canada]
2       [resident, asked, shelter, place, notified, of...
3        [people, receive, evacuation, order, california]
4          [got, sent, photo, ruby, smoke, pours, school]
                              ...                        
3258    [earthquake, safety, los, angeles, safety, fas...
3259    [storm, ri, worse, last, hurricane, city, othe...
3260                   [green, line, derailment, chicago]
3261       [meg, issue, hazardous, weather, outlook, hwo]
3262              [activated, municipal, emergency, plan]
Name: text, Length: 10876, dtype: object

In [85]:
w2v_model = gensim.models.Word2Vec(sentence2,min_count=1,vector_size=100)

In [86]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [87]:
most_similar(w2v_model,['reason'])

,reason,cos
0,think,0.998912
1,death,0.998795
2,weapon,0.998767
3,via,0.998760
4,got,0.998706
5,not,0.998703
6,fatality,0.998698
7,girl,0.998675
8,hostage,0.998674
9,one,0.998655


In [89]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393395 sha256=2ac59dc5b6acb96c72afa30ce292d01128957b295a5a4018808e59b9ba8230a2
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [90]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

In [91]:
ft.get_nearest_neighbors('hello')

[(0.7143728733062744, 'hellow'),
 (0.7095366716384888, 'hello.'),
 (0.703833818435669, 'hi'),
 (0.6944323182106018, 'hullo'),
 (0.6912142038345337, 'hello-'),
 (0.663975715637207, 'Hello'),
 (0.6563249230384827, 'hello.I'),
 (0.6529381275177002, 'howdy'),
 (0.6283847689628601, 'hellooo'),
 (0.6235803365707397, 'hellooooo')]